In [18]:
import numpy as np

def _gen_stock_vec(nb, h):
    s0 = 100
    sigma = 0.5
    up = np.exp(sigma * np.sqrt(2 * h))
    down = 1 / up  # down movement to force a "recombining tree"

    vec_u = up * np.ones(nb)
    np.cumprod(vec_u, out=vec_u)  # Computing u, u^2, u^3....u^nb

    vec_d = down * np.ones(nb)
    np.cumprod(vec_d, out=vec_d)  # Computing d, d^2, d^3....d^nb

    res = np.concatenate(
        (vec_d[::-1], [1.0], vec_u)
    )  # putting together the last period tree underlyings
    res *= s0
    return res

def price(
    nb_steps,
    r,
    sigma,
    T,
    K
):  # For now, the only input to the function is the number of steps for the tree, N.
    # Define parameters
    r = 0
    sigma = 0.5
    T = 1
    K = 130
    h = T / nb_steps  # This would be our 'dt' from previous examples
    discount = np.exp(-r * h)  # Define discount factor for simplicity later on

    # Define risk-neutral probabilities:
    pu = (
        (np.exp(r * h / 2) - np.exp(-sigma * np.sqrt(h / 2)))
        / (np.exp(sigma * np.sqrt(h / 2)) - np.exp(-sigma * np.sqrt(h / 2)))
    ) ** 2
    pd = (
        (-np.exp(r * h / 2) + np.exp(sigma * np.sqrt(h / 2)))
        / (np.exp(sigma * np.sqrt(h / 2)) - np.exp(-sigma * np.sqrt(h / 2)))
    ) ** 2
    pm = 1 - pu - pd
    print(pu, pd, pm)
    # This would be our underlying evolution (Note we are using the function from before!)
    s = _gen_stock_vec(nb_steps, h)

    # Define Payoff (in this case, European Call Option)
    final_payoff = np.maximum(s - K, 0)
    nxt_vec_prices = final_payoff

    # Proceed with iterations for the calculation of payoffs
    for i in range(1, nb_steps + 1):
        vec_stock = _gen_stock_vec(nb_steps - i, h)
        expectation = np.zeros(vec_stock.size)

        for j in range(expectation.size):
            tmp = nxt_vec_prices[j] * pd
            tmp += nxt_vec_prices[j + 1] * pm
            tmp += nxt_vec_prices[j + 2] * pu

            expectation[j] = tmp
        # Discount option payoff!
        nxt_vec_prices = discount * expectation

    return nxt_vec_prices[
        0
    ]  # Notice here we only 'return' the expected discounted value of the option at t=0, that is, the price of the call option!

In [19]:
import numpy as np

def _gen_stock_vec(S0, sigma, nb, h):
    up = np.exp(sigma * np.sqrt(2 * h))
    down = 1 / up  # down movement to force a "recombining tree"

    vec_u = up * np.ones(nb)
    np.cumprod(vec_u, out=vec_u)  # Computing u, u^2, u^3....u^nb

    vec_d = down * np.ones(nb)
    np.cumprod(vec_d, out=vec_d)  # Computing d, d^2, d^3....d^nb

    res = np.concatenate(
        (vec_d[::-1], [1.0], vec_u)
    )  # putting together the last period tree underlyings
    res *= S0
    return res

def price(
    S0,
    r,
    sigma,
    T,
    K,
    nb_steps,
    option_type="call",
    option_style="european"
):
    h = T / nb_steps  # This would be our 'dt' from previous examples
    discount = np.exp(-r * h)  # Define discount factor for simplicity later on

    # Define risk-neutral probabilities:
    pu = (
        (np.exp(r * h / 2) - np.exp(-sigma * np.sqrt(h / 2)))
        / (np.exp(sigma * np.sqrt(h / 2)) - np.exp(-sigma * np.sqrt(h / 2)))
    ) ** 2
    pd = (
        (-np.exp(r * h / 2) + np.exp(sigma * np.sqrt(h / 2)))
        / (np.exp(sigma * np.sqrt(h / 2)) - np.exp(-sigma * np.sqrt(h / 2)))
    ) ** 2
    pm = 1 - pu - pd
    print(pu, pd, pm)

    # This would be our underlying evolution
    s = _gen_stock_vec(S0, sigma, nb_steps, h)

    # Define Payoff (for call or put option)
    if option_type == "call":
        final_payoff = np.maximum(s - K, 0)
    elif option_type == "put":
        final_payoff = np.maximum(K - s, 0)

    nxt_vec_prices = final_payoff

    # Proceed with iterations for the calculation of payoffs
    for i in range(1, nb_steps + 1):
        vec_stock = _gen_stock_vec(S0, sigma, nb_steps - i, h)
        expectation = np.zeros(vec_stock.size)

        for j in range(expectation.size):
            tmp = nxt_vec_prices[j] * pd
            tmp += nxt_vec_prices[j + 1] * pm
            tmp += nxt_vec_prices[j + 2] * pu

            # If American option, check for early exercise
            if option_style == "american":
                if option_type == "call":
                    tmp = max(tmp, vec_stock[j] - K)
                elif option_type == "put":
                    tmp = max(tmp, K - vec_stock[j])

            expectation[j] = tmp

        # Discount option payoff!
        nxt_vec_prices = discount * expectation

    return nxt_vec_prices[
        0
    ]  # Notice here we only 'return' the expected discounted value of the option at t=0, that is, the price of the option!

# Example usage
S0 = 100   # initial stock price
K = 90    # strike price
T = 1      # time to maturity in years
r = 0.0   # risk-free rate
sigma = 0.3  # volatility
nb_steps = 1  # number of steps in the trinomial tree

# Calculate prices
call_price_european = price(S0, r, sigma, T, K, nb_steps, option_type="call", option_style="european")
put_price_american = price(S0, r, sigma, T, K, nb_steps, option_type="put", option_style="american")

print("European Call Option Price: ", round(call_price_european, 2))
print("American Put Option Price: ", round(put_price_american, 2))

0.19995651425667543 0.30562656562099344 0.49441692012233107
0.19995651425667543 0.30562656562099344 0.49441692012233107
European Call Option Price:  17.51
American Put Option Price:  7.51


In [21]:
100*np.exp(0.2*5)

271.8281828459045